In [114]:
from bs4 import BeautifulSoup as bs
import urllib
import re
import string
import pandas as pd
from operator import itemgetter

In [173]:
bi = pd.read_csv('../British_Invasion/Data/List_of_British_Invasion_artists.csv')
print len(bi.artist.values)

bi2 = pd.read_csv('../British_Invasion/Data/List_of_Second_British_Invasion_artists.csv')
print len(bi2.artist.values)

45
51


In [177]:
def url(artist):
    base = "http://www.lyricsfreak.com/"
    name = artist.lower().strip().translate(None, string.punctuation).split(" ")
    if '' in name:
        name.remove('')
    init = name[0][0]
    search_name = "+".join(name)
    URL = base + init + "/" + search_name + "/"
    return URL

def fetch(artist):
    base = "http://www.lyricsfreak.com"
    song_list = []
    URL = url(artist)
    page = urllib.urlopen(URL)
    soup = bs(page, "lxml")
    songs = soup.findAll("table", attrs={"name": "song"})[0].findAll("tr")
    for s in songs:
        for n in s.findAll("td", attrs={"class": "colfirst"}):
            name = re.sub(" Lyrics","",n.find('a').attrs["title"])
            lyric_link = base + n.find('a').attrs["href"]
            song_list.append({name:lyric_link})
    return song_list

def fetch_all_artists(artist_list):
    lists = {}
    for i, artist in enumerate(artist_list):
        # print artist
        # print "\r{}%".format(100.0*(i+1)/len(artist_list)),
        song_list = []
        if artist[:3].lower() == 'the':
            try:
                song_list = fetch(artist)
            except IndexError:
                pass
            try:
                song_list += fetch(artist[4:])
            except IndexError:
                pass
        elif 'and' in artist.split(" "):
            try:
                song_list = fetch(artist)
            except IndexError:
                pass
            try:
                song_list += fetch(re.sub(' and ', ' & ', artist))
            except IndexError:
                pass
        else:
            try:
                song_list = fetch(artist)
            except:
                print "[Warning!] {}: Not Exist".format(artist)
        lists[artist] = song_list
    return lists

def ly_correct_url(URL):
    # input: lyric_page_url
    # output: lyric_correct_url
    page = urllib.urlopen(URL)
    soup = bs(page, 'lxml')
    base = 'http://www.lyricsfreak.com'
    return base + soup.find_all('a', attrs={'class': 'lb-correct'})[0].attrs['href']

def fetch_lyrics(URL):
    # input: lyric_page_url
    URL2 = ly_correct_url(URL)
    page = urllib.urlopen(URL2)
    soup = bs(page, 'lxml')
    lyric_text = soup.findAll('textarea', attrs={'name': 'lyrics'})[0].text
    return lyric_text

def fetch_one_artist_all_lyrics(artist):
    li =  [artist] #;print li
    d = fetch_all_artists(li) #;print d
    song_list = d[artist]
    lyrics_list = {} # {name: lyrics}
    for i, song in enumerate(song_list):
        # print '\r{}%'.format(100.0*(i+1)/len(song_list)),
        name = song.keys()[0] #;print name
        url = song.values()[0] #;print url
        lyrics = fetch_lyrics(url)
        lyrics_list[name] = lyrics
    return lyrics_list

def fetch_all_artists_all_lyrics(artist_list):
    artists_songs_lyrics = {} # {artist: {song: lyrics}}
    for i, artist in enumerate(artist_list):
        songs_lyrics = fetch_one_artist_all_lyrics(artist)
        artists_songs_lyrics[artist] = songs_lyrics
        print '\r{}%'.format(100.0*(i+1)/len(artist_list)),
    return artists_songs_lyrics

## Test 
#### 1 - 9

In [175]:
res = fetch_all_artists_all_lyrics(bi.artist.values[1:10])

300.0%


In [183]:
for k,v in res.iteritems():
    print k, len(v), '\b',
    num = 0
    for s, ly in v.iteritems():
        if ly != u'': # Not Exist!
            num += 1
    print num

Chad & Jeremy 17  17
Donovan 197  197
The Beatles 202  201
Petula Clark 69  69
The Spencer Davis Group 12  8
Dave Dee, Dozy, Beaky, Mick & Tich 9  9
The Dave Clark Five 35  20
Cilla Black 14  14
Cream 45  45


## Fetch All - slow - Not Test yet

In [ ]:
# not test yet
# slow .. slow .. slow

BI1_lyrics = fetch_all_artists_all_lyrics(bi.artist.values)
BI2_lyrics = fetch_all_artists_all_lyrics(bi2.artist.values)

<br/><br/><br/><br/><br/><br/>

In [ ]:
# l = ['Wayne Fontana and the Mindbenders', 'Billy J. Kramer & The Dakotas', 'Dave Dee, Dozy, Beaky, Mick & Tich']
# res = fetch_all_artists_all_lyrics(l)
# res['Billy J. Kramer & The Dakotas']['Bad To Me']

In [51]:
# Fail to fetch lyrics directly from Lyrics Page
# page = urllib.urlopen('http://www.lyricsfreak.com/w/who/another+tricky+day_20146468.html')
# soup = bs(page, 'lxml')
# a = soup.find_all('div', attrs={'class': 'content song-content floatfix'})[0]
# b = a.find_all('div', attrs={'id': 'content'})[0].get_text()
# b

In [151]:
# l = ['Wayne Fontana and the Mindbenders', 'Billy J. Kramer & The Dakotas', 'Dave Dee, Dozy, Beaky, Mick & Tich']
# d = fetch_all_artists(l)

In [164]:
#song_list = d['Wayne Fontana and the Mindbenders']
#fetch_one_artist_all_lyrics(song_list)
#fetch_one_artist_all_lyrics('Wayne Fontana and the Mindbenders')

In [72]:
# ly = fetch_lyrics('http://www.lyricsfreak.com/lyrics_correct.php?song=224460')
# ly2 = fetch_lyrics('http://www.lyricsfreak.com/lyrics_correct.php?song=224463')

In [84]:
# page = urllib.urlopen('http://www.lyricsfreak.com/w/who/another+tricky+day_20146468.html')
# soup = bs(page, 'lxml')
# base = 'http://www.lyricsfreak.com'
# page2 = base + soup.find_all('a', attrs={'class': 'lb-correct'})[0].attrs['href']
# fetch_lyrics(page2)

In [150]:
# fetch_lyrics('http://www.lyricsfreak.com/w/who/another+tricky+day_20146468.html')
# fetch_lyrics('http://www.lyricsfreak.com/w/who/905_20146806.html')